In [2]:
#Install/import necessary packages
!pip install mtgsdk
from mtgsdk import Card
from bs4 import BeautifulSoup

import requests
import pandas as pd
import json
import pickle
from pandas.io.json import json_normalize

# CARD DATA SCRAPING 

If you don't have the data/modernCardsUnprocessed.pickle file, run these cells:

In [8]:
#Get all modern cards
cards = Card.where(gameFormat='modern').all()
cardDicts = [c.__dict__ for c in cards]
cardsDf = pd.DataFrame(cardDicts)

In [9]:
with open('data/modernCardsUnprocessed.pickle','wb') as pickle_out:
    pickle.dump(cardsDf,pickle_out)

Otherwise, run these cell:

In [11]:
#Open unprocessed data
with open('data/modernCardsUnprocessed.pickle','rb') as pickle_in:
    cardsDf = pickle.load(pickle_in)

In [12]:
#Filter out the lands
types = cardsDf['types']

nonlandFilter = ["Land" not in t for t in types]

nonLands = pd.DataFrame(cardsDf.loc[nonlandFilter])

In [13]:
#This will fix list columns with null values and join lists by commas
def fixListCol(el):
    if el == None:
        return ''
    elif type(el) == list:
        return ','.join(el)
    return el

In [14]:
# Apply the fix to relevant columns
nonLands['supertypes'] = nonLands['supertypes'].apply(fixListCol)
nonLands['types'] = nonLands['types'].apply(fixListCol)
nonLands['subtypes'] = nonLands['subtypes'].apply(fixListCol)
nonLands['color_identity'] = nonLands['color_identity'].apply(fixListCol)


In [32]:
#Select the relevant columns
cardsDfFinal = nonLands[['artist','cmc','color_identity','layout','loyalty','mana_cost','name','power','rarity','subtypes','supertypes','text','timeshifted','toughness','types','set']]

print(cardsDfFinal.head())

            artist  cmc color_identity  layout  loyalty  mana_cost  \
0      Warren Mahy    4              R  normal      NaN     {3}{R}   
16  Douglas Shuler    5              W  normal      NaN  {3}{W}{W}   
17  Trevor Claxton    4              G  normal      NaN  {2}{G}{G}   
18            Izzy    3            W,U  normal      NaN  {1}{W}{U}   
19    Steve Argyle    2              W  normal      NaN     {1}{W}   

                           name power   rarity       subtypes supertypes  \
0             Steamflogger Boss     3     Rare  Goblin,Rigger              
16                  Serra Angel     4  Special          Angel              
17                    Vengevine     4  Special      Elemental              
18         Geist of Saint Traft     2  Special  Spirit,Cleric  Legendary   
19  Thalia, Guardian of Thraben     2  Special  Human,Soldier  Legendary   

                                                 text timeshifted toughness  \
0   Other Riggers you control get +1/+0 and

In [10]:
cardsDfFinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20018 entries, 0 to 23042
Data columns (total 19 columns):
artist            20018 non-null object
border            3 non-null object
cmc               20018 non-null int64
color_identity    20018 non-null object
layout            20018 non-null object
loyalty           168 non-null float64
mana_cost         19913 non-null object
name              20018 non-null object
power             10978 non-null object
rarity            20018 non-null object
release_date      777 non-null datetime64[ns]
source            1065 non-null object
subtypes          20018 non-null object
supertypes        20018 non-null object
text              19556 non-null object
timeshifted       236 non-null object
toughness         10978 non-null object
types             20018 non-null object
set               20018 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(16)
memory usage: 3.7+ MB


In [18]:
print(cardsDfFinal.loc[pd.isnull(cardsDfFinal['mana_cost'])].head())

               artist  cmc color_identity        layout  loyalty mana_cost  \
83   Christopher Rush    0                       normal      NaN      None   
104     Ryan Pancoast    2              G  double-faced      NaN      None   
106        Kev Walker    5              B  double-faced      NaN      None   
603     Karl Kopinski    2              U  double-faced      NaN      None   
605   Anthony Palumbo    4              R  double-faced      NaN      None   

                      name power   rarity  \
83             Lotus Bloom  None  Special   
104         Howlpack Alpha     3  Special   
106     Archdemon of Greed     9  Special   
603  Ludevic's Abomination    13  Special   
605   Tovolar's Magehunter     5  Special   

                                              source       subtypes  \
83       Time Spiral Prerelease participation bonus.                  
104         Innistrad Prerelease participation bonus       Werewolf   
106    Dark Ascension Prerelease participation 

In [33]:
#Save the data
with open('data/modernCardsFinal.pickle','wb') as pickle_out:
    pickle.dump(cardsDfFinal,pickle_out)

In [4]:
with open('data/modernCardsFinal.pickle','rb') as pickle_in:
    cardsDfFinal = pickle.load(pickle_in)

# PRICE SCRAPING

If the file already exists in the data directory:

In [20]:
with open('data/modernPrices.pickle','rb') as pickle_in:
    pricesDF = pickle.load(pickle_in)

In [22]:
print(pricesDF.loc[pricesDF['name']=='Howlpack Alpha'])

Empty DataFrame
Columns: [name, price, set, setname]
Index: []


If the data needs to be scraped:

In [5]:
#This will scrape the prices from the formats included in the given format parameter (i.e. 'Modern' will scrape both Standard and Modern)
def scrapeSelectPage(url,formt):
    #Get the page
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc,'lxml')
    
    #Get the format blocks from the page
    formats = soup.find_all('div',class_='priceList-setMenu')
    pricesDF = pd.DataFrame()
    for form in formats:
        pricesDF = pricesDF.append(scrapeFormat(form))
        
        #Check if we've reached the desired format
        if 'priceList-setMenu-'+formt in form.attrs['class']:
            break
        
    return pricesDF

#Scrapes a format div from the /prices/select page
def scrapeFormat(soup):
    #Select the first li element in the format's div
    element = soup.li
    
    #Skip the first "Quick Prices" block
    element = element.find_next_sibling('li',class_='priceList-setMenu-header')
    
    #Create empty dataframe to append to
    formatDF = pd.DataFrame()
    
    #Append the Dataframes obtained by scraping each set
    while(element.find_next_sibling('li') != None):
        #Iterate
        element = element.find_next_sibling('li')
        
        #The set elements don't have classes associated with them
        if 'class' not in element.attrs:
            #Scrape the cards in the set's page and add the name of the set
            setDF = scrapeSet('https://www.mtggoldfish.com'+element.a.attrs['href']+'#paper')
            setDF['setname'] = element.get_text().strip()
            
            formatDF = formatDF.append(setDF)
            
    return formatDF

#Scrapes all of the prices from the given set page
def scrapeSet(url):
    #Get the page
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc,'lxml')
    
    #Select the cards table
    element = soup.find('table',class_='tablesorter-bootstrap-popover-online').tbody
    
    #Build a list that will be filled with dictionaries to be converted to a DataFrame
    cardList = []
    
    #Loop through rows and scrape the name and price
    for row in element.find_all('tr'):
        #Get the td elements
        row = row.find_all('td')
        
        cardList.append({'name':row[0].a.get_text().strip(),'price':row[3].get_text().strip(),'set':row[1].get_text()})
    
    return pd.DataFrame(cardList)

In [6]:
pricesDF = scrapeSelectPage('https://www.mtggoldfish.com/prices/select','Modern')

                       name  price  set           setname
0        Rekindling Phoenix  20.64  RIX  Rivals of Ixalan
1  Kumena, Tyrant of Orazca   8.02  RIX  Rivals of Ixalan
2  Angrath, Minotaur Pirate   6.00  RIX  Rivals of Ixalan
3   Vraska, Scheming Gorgon   5.00  RIX  Rivals of Ixalan
4          Jadelight Ranger   3.83  RIX  Rivals of Ixalan
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13816 entries, 0 to 285
Data columns (total 4 columns):
name       13816 non-null object
price      13816 non-null object
set        13816 non-null object
setname    13816 non-null object
dtypes: object(4)
memory usage: 539.7+ KB


In [8]:
with open('data/modernPrices.pickle','wb') as pickle_out:
    pickle.dump(pricesDF,pickle_out)

Look at the DF:

In [10]:
print(pricesDF.head())
pricesDF.info()

                       name  price  set           setname
0        Rekindling Phoenix  20.64  RIX  Rivals of Ixalan
1  Kumena, Tyrant of Orazca   8.02  RIX  Rivals of Ixalan
2  Angrath, Minotaur Pirate   6.00  RIX  Rivals of Ixalan
3   Vraska, Scheming Gorgon   5.00  RIX  Rivals of Ixalan
4          Jadelight Ranger   3.83  RIX  Rivals of Ixalan
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13816 entries, 0 to 285
Data columns (total 4 columns):
name       13816 non-null object
price      13816 non-null object
set        13816 non-null object
setname    13816 non-null object
dtypes: object(4)
memory usage: 539.7+ KB


# MELD

In [34]:
#Merge the card info and card prices dataframes
cardsWithPrices = pricesDF.merge(cardsDfFinal,on=['name','set'])

#Set the index to the name and set
cardsWithPrices = cardsWithPrices.set_index(['set','name']).sort_index()

In [35]:
#Convert to useful datatypes
cardsWithPrices['cmc'] = pd.to_numeric(cardsWithPrices['cmc'])
cardsWithPrices['loyalty'] = pd.to_numeric(cardsWithPrices['loyalty'])
cardsWithPrices['price'] = pd.to_numeric(cardsWithPrices['price'])
cardsWithPrices['rarity'] = cardsWithPrices['rarity'].astype('category')
cardsWithPrices['layout'] = cardsWithPrices['layout'].astype('category')

In [51]:
#Manually adding a manacost to Suspend and no-cost Splice cards as their ability's respective cost
cardsWithPrices.loc[(slice(None),'Evermind'),'mana_cost'] = '{1}{U}'
cardsWithPrices.loc[(slice(None),'Evermind'),'cmc'] = 2

cardsWithPrices.loc[(slice(None),'Wheel of Fate'),'mana_cost'] = '{1}{R}'
cardsWithPrices.loc[(slice(None),'Wheel of Fate'),'cmc'] = 2

cardsWithPrices.loc[(slice(None),'Restore Balance'),'mana_cost'] = '{W}'
cardsWithPrices.loc[(slice(None),'Restore Balance'),'cmc'] = 1

cardsWithPrices.loc[(slice(None),'Living End'),'mana_cost'] = '{2}{B}{B}'
cardsWithPrices.loc[(slice(None),'Living End'),'cmc'] = 4

cardsWithPrices.loc[(slice(None),'Ancestral Vision'),'mana_cost'] = '{U}'
cardsWithPrices.loc[(slice(None),'Ancestral Vision'),'cmc'] = 1

cardsWithPrices.loc[(slice(None),'Lotus Bloom'),'mana_cost'] = '{0}'
cardsWithPrices.loc[(slice(None),'Lotus Bloom'),'cmc'] = 0

#Drop the two transformed cards that slipped through the cracks
cardsWithPrices = cardsWithPrices.drop([('ISD','Howlpack of Estwald'),('EMN','Howling Chorus')])

In [58]:
#Fill empty text 
cardsWithPrices['text'] = cardsWithPrices['text'].fillna('')

In [60]:
#Everything is good to go!
cardsWithPrices.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12730 entries, (10E, Abundance) to (ZEN, Zendikar Farguide)
Data columns (total 16 columns):
price             12730 non-null float64
setname           12730 non-null object
artist            12730 non-null object
cmc               12730 non-null int64
color_identity    12730 non-null object
layout            12730 non-null category
loyalty           112 non-null float64
mana_cost         12730 non-null object
power             7060 non-null object
rarity            12730 non-null category
subtypes          12730 non-null object
supertypes        12730 non-null object
text              12730 non-null object
timeshifted       233 non-null object
toughness         7060 non-null object
types             12730 non-null object
dtypes: category(2), float64(2), int64(1), object(11)
memory usage: 1.5+ MB


In [61]:
#Just out of curiosity, let's see the top 20 priced cards
print(cardsWithPrices.sort_values(by='price', ascending=False)['price'].head(20))

set  name                      
SOM  Mox Opal                      57.98
MM2  Mox Opal                      56.13
ISD  Liliana of the Veil           53.64
MM3  Liliana of the Veil           53.27
HOU  The Scarab God                38.05
NPH  Karn Liberated                37.81
MM2  Karn Liberated                36.52
5DN  Engineered Explosives         35.88
MMA  Engineered Explosives         35.82
8ED  Ensnaring Bridge              33.52
KLD  Chandra, Torch of Defiance    30.43
EMN  Liliana, the Last Hope        28.58
NPH  Surgical Extraction           27.56
MM2  Surgical Extraction           27.53
8ED  Blood Moon                    25.95
MMA  Chalice of the Void           25.35
MRD  Chalice of the Void           25.25
MMA  Tarmogoyf                     24.38
FUT  Tarmogoyf                     24.00
MM3  Tarmogoyf                     23.41
Name: price, dtype: float64
